In [1]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import json
from pymongo import MongoClient
from pymongo.errors import ConnectionError, BulkWriteError

# --- Configuration ---
MONGODB_URI = "mongodb://localhost:27017/"  # Default local MongoDB URI
DATABASE_NAME = "MyLocalDB"                # Your database name
COLLECTION_NAME = "Products"               # Your collection name
JSON_FILE_PATH = "data.json"               # The name of your JSON file
# --- End Configuration ---

def import_json_to_mongodb(uri, db_name, collection_name, file_path):
    """
    Connects to MongoDB, reads a JSON file, and inserts the data.
    """
    client = None
    try:
        # 1. Connect to MongoDB
        client = MongoClient(uri)
        # The ismaster command is cheap and does not require auth.
        client.admin.command('ping') 
        print(f"✅ Successfully connected to MongoDB at {uri}")

        # 2. Get Database and Collection
        db = client[db_name]
        collection = db[collection_name]
        print(f"➡️ Targeting database: {db_name}, collection: {collection_name}")

        # 3. Load JSON data from file
        with open(file_path, 'r', encoding='utf-8') as file:
            file_data = json.load(file)
            print(f"✅ Successfully loaded data from {file_path}")

        # 4. Insert data based on its structure
        if isinstance(file_data, list):
            # If the JSON file is a list of documents, use insert_many()
            result = collection.insert_many(file_data)
            print(f"✨ Inserted {len(result.inserted_ids)} documents using insert_many().")
            print("--- First 3 IDs: ", result.inserted_ids[:3])
        elif isinstance(file_data, dict):
            # If the JSON file is a single document (dictionary), use insert_one()
            result = collection.insert_one(file_data)
            print(f"✨ Inserted 1 document using insert_one(). ID: {result.inserted_id}")
        else:
            print(f"❌ Error: JSON file content is neither a list nor a dictionary. Type: {type(file_data)}")
            return

    except ConnectionError:
        print(f"❌ CONNECTION ERROR: Could not connect to MongoDB. Ensure your MongoDB server is running at {uri}.")
    except FileNotFoundError:
        print(f"❌ FILE ERROR: The file '{file_path}' was not found.")
    except json.JSONDecodeError as e:
        print(f"❌ JSON ERROR: Failed to decode JSON from file. Check file syntax. Error: {e}")
    except BulkWriteError as e:
        # Catches errors like duplicate keys if you are inserting documents with specific _id fields
        print(f"⚠️ BULK WRITE ERROR: Some documents failed to insert. {e.details}")
    except Exception as e:
        print(f"❌ An unexpected error occurred: {e}")
    finally:
        # 5. Close the connection
        if client:
            client.close()
            print("➡️ MongoDB connection closed.")

# Run the function
import_json_to_mongodb(MONGODB_URI, DATABASE_NAME, COLLECTION_NAME, JSON_FILE_PATH)